In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain

from langchain.prompts import PromptTemplate

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory

from langchain.agents import load_tools

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
# llm = OpenAI(temperature=0, verbose=True)
llm = ChatOpenAI(model_name='gpt-4',temperature=0)

In [5]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [6]:
from langchain.tools import Tool, BaseTool
from pydantic.v1 import BaseModel, Field
import math
from scipy.stats import norm

def rw_corp(Input_String: str) -> str:

    """ 
    calculates the risk weight from input parameters 
    PD, LGD, MATURITY, F_LARGE_FIN and SIZE which are fed into the formula in this particular order 
    """
    
    PD_s, LGD_s, MATURITY_s, SIZE_s, F_LARGE_FIN_s = Input_String.split(",")

    PD = float(eval(PD_s.split(":")[1].strip()))
    LGD = float(eval(LGD_s.split(":")[1].strip()))
    MATURITY = float(eval(MATURITY_s.split(":")[1].strip()))
    SIZE = float(eval(SIZE_s.split(":")[1].strip()))
    F_LARGE_FIN = (F_LARGE_FIN_s.split(":")[1].strip())
       
    pd_final = max(0.0003, PD)
    size_final = max(5, min(SIZE, 50))    
    r0 = (0.12 * (1.0 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0))) + \
        (0.24 * (1.0 - (1 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0)))) - \
        (0.04 * (1.0 - (size_final - 5.0) / 45.0))
    if F_LARGE_FIN == 'Y':
        r = r0 * 1.25
    else:
        r = r0
    b = (0.11852 - 0.05478 * math.log(pd_final)) ** 2
    ma = ((1 - 1.5 * b) ** -1) * (1 + (MATURITY - 2.5) * b)    
    rw = ((LGD * norm.cdf((1 - r) ** -0.5 * norm.ppf(pd_final) + (r / (1 - r)) ** 0.5 * norm.ppf(0.999)) - pd_final * LGD) * ma) * 12.5 * 1.06
    return rw  


class RWInput(BaseModel):
    Input_String: str = Field(description='This is a string that contains values for the input parameters \
                              PD, LGD, MATURITY, SIZE and F_LARGE_FIN which are fed into the formula in this particular order ')
    
RWTool = Tool.from_function(
    func=rw_corp,
    name="RWTool",
    description="""
    This is a custom tool that calculates the risk weight from a set of input parameters:
        PD - Probability of Default,
        LGD - Loss Given Default,
        MATURITY - Remaining maturity of the loan in years,
        SIZE - The size of the client in MEUR, usually this is the client's turnover, 
        F_LARGE_FIN - If 'Y' the client is a Large Financial Institution        
        """,
    args_schema=RWInput
)

In [7]:
tools.append(RWTool)

In [8]:
db = SQLDatabase.from_uri("sqlite:///./portfolio_data.db",
                         include_tables=['corporate_portfolio'],
                         sample_rows_in_table_info=2
)

In [9]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# custom_suffix = """
# Use this math tool for performing mathematical operation on sql querry results.
# Example of such question: What is the difference between two sums obtained via a sql querry.
# """
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    extra_tools=tools,
  
)

### sql

In [9]:
# output = agent.run("What are total RWA and EAD for reporting date 31-12-2022?")

In [10]:
# output = agent.run("What is the change in total RWA from reporting date 2022-12-31 to reporting date 2023-03-31?")

In [18]:
# output = agent.run("What is the total exposure of only those loan ids that have status from Performing on reporting date 2022-12-31 and status Non Performing on reporting date 2023-03-31?")

#### sql + rw

In [1]:
# output = agent.run("Read the information in table corporate_portfolio for loan id 1439964799kgeq68 and for reporting date 31-12-2022")

In [2]:
# output = agent.run("What will be the RW if PD doubles")

In [3]:
# output = agent.run("What about if LGD doubles")

In [4]:
# output = agent.run("And if MATURITY becomes 6.23")

In [5]:
# output = agent.run("What if all three parameters change like that?")

In [6]:
# output = agent.run("Start over again and read the all information in table corporate_portfolio for loan id 1439964799kgeq68 but for reporting date 2023-03-31")

In [7]:
# output = agent.run("What is the change in PD for loan id 1439964799kgeq68 between reporting date 2022-12-31 and 2023-03-31")

In [8]:
# output = agent.run("Read the information for ACCOUNT_NUMBER = '1851285872eypk20' in rwa_data table and then calculate the RW if LGD doubles and PD becomes 0.2")